In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

In [2]:
def load_data(path):
    df = pd.read_json(path)
    bert = np.array(df['bert_cls'].tolist())
    roberta = np.array(df['roberta_cls'].tolist())
    y = df['fake'].values
    return bert, roberta, y

# Concatenate

In [3]:
def build_model():
    bert_input = layers.Input(shape=(768,))
    roberta_input = layers.Input(shape=(768,))

    # BERT branch
    bert_branch = layers.Dense(256, activation='relu')(bert_input)
    bert_branch = layers.Dropout(0.3)(bert_branch)
    bert_branch = layers.Dense(128, activation='relu')(bert_branch)

    # RoBERTa branch
    roberta_branch = layers.Dense(256, activation='relu')(roberta_input)
    roberta_branch = layers.Dropout(0.3)(roberta_branch)
    roberta_branch = layers.Dense(128, activation='relu')(roberta_branch)

    # Concatenate and output
    combined = layers.Concatenate()([bert_branch, roberta_branch])
    combined = layers.Dropout(0.3)(combined)
    output = layers.Dense(1, activation='sigmoid')(combined)

    model = models.Model(inputs=[bert_input, roberta_input], outputs=output)
    model.compile(optimizer=optimizers.Adam(1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [4]:
def train_model(bert, roberta, y):
    X_train_bert, X_val_bert, y_train, y_val = train_test_split(
        bert, y, test_size=0.2, stratify=y, random_state=42)
    X_train_roberta, X_val_roberta, _, _ = train_test_split(
        roberta, y, test_size=0.2, stratify=y, random_state=42)

    model = build_model()

    early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
        [X_train_bert, X_train_roberta], y_train,
        validation_data=([X_val_bert, X_val_roberta], y_val),
        epochs=100,
        batch_size=64,
        callbacks=[early_stop]
    )

    y_pred_prob = model.predict([X_val_bert, X_val_roberta])
    y_pred = (y_pred_prob > 0.5).astype(int)

    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)
    rec = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1 Score : {f1:.4f}")

    return model

In [45]:
# 실행
bert, roberta, y = load_data("emb_cls.json")
model = train_model(bert, roberta, y)

Epoch 1/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6717 - loss: 0.6008 - val_accuracy: 0.7198 - val_loss: 0.5437
Epoch 2/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7183 - loss: 0.5502 - val_accuracy: 0.7240 - val_loss: 0.5352
Epoch 3/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7229 - loss: 0.5408 - val_accuracy: 0.7271 - val_loss: 0.5314
Epoch 4/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7261 - loss: 0.5344 - val_accuracy: 0.7296 - val_loss: 0.5281
Epoch 5/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7290 - loss: 0.5318 - val_accuracy: 0.7310 - val_loss: 0.5255
Epoch 6/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7366 - loss: 0.5270 - val_accuracy: 0.7328 - val_loss: 0.5238
Epoch 7/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7365 - loss: 0.5232 - val_accuracy: 0.7352 - val_loss: 0.5223
Epoch 8/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7384 - loss: 0

# Multiply

In [10]:
from tensorflow.keras.layers import Multiply
def build_model_multiply():
    bert_input = layers.Input(shape=(768,))
    roberta_input = layers.Input(shape=(768,))

    # BERT branch
    bert_branch = layers.Dense(256, activation='relu')(bert_input)
    bert_branch = layers.Dropout(0.3)(bert_branch)
    bert_branch = layers.Dense(128, activation='relu')(bert_branch)

    # RoBERTa branch
    roberta_branch = layers.Dense(256, activation='relu')(roberta_input)
    roberta_branch = layers.Dropout(0.3)(roberta_branch)
    roberta_branch = layers.Dense(128, activation='relu')(roberta_branch)

    # Concatenate and output
    combined = Multiply()([bert_branch, roberta_branch])
    combined = layers.Dropout(0.3)(combined)
    output = layers.Dense(1, activation='sigmoid')(combined)

    model = models.Model(inputs=[bert_input, roberta_input], outputs=output)
    model.compile(optimizer=optimizers.Adam(1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [47]:
def train_model_multiply(bert, roberta, y):
    X_train_bert, X_val_bert, y_train, y_val = train_test_split(
        bert, y, test_size=0.2, stratify=y, random_state=42)
    X_train_roberta, X_val_roberta, _, _ = train_test_split(
        roberta, y, test_size=0.2, stratify=y, random_state=42)

    model_multiply = build_model()

    early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model_multiply.fit(
        [X_train_bert, X_train_roberta], y_train,
        validation_data=([X_val_bert, X_val_roberta], y_val),
        epochs=100,
        batch_size=64,
        callbacks=[early_stop]
    )

    y_pred_prob = model_multiply.predict([X_val_bert, X_val_roberta])
    y_pred = (y_pred_prob > 0.5).astype(int)

    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)
    rec = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1 Score : {f1:.4f}")

    return model_multiply

In [58]:
# 실행
bert, roberta, y = load_data("emb_cls.json")
model= train_model_multiply(bert, roberta, y)

Epoch 1/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6721 - loss: 0.6020 - val_accuracy: 0.7194 - val_loss: 0.5445
Epoch 2/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7156 - loss: 0.5510 - val_accuracy: 0.7240 - val_loss: 0.5356
Epoch 3/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7233 - loss: 0.5418 - val_accuracy: 0.7253 - val_loss: 0.5313
Epoch 4/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7277 - loss: 0.5361 - val_accuracy: 0.7286 - val_loss: 0.5282
Epoch 5/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7300 - loss: 0.5307 - val_accuracy: 0.7285 - val_loss: 0.5268
Epoch 6/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7336 - loss: 0.5272 - val_accuracy: 0.7318 - val_loss: 0.5247
Epoch 7/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7351 - loss: 0.5234 - val_accuracy: 0.7321 - val_loss: 0.5235
Epoch 8/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7368 - loss: 0